# class RNA

In [1]:
##############
# class RNA #
##############
# has string attributes name, id (most likely from a db), species, and sequence (5'->3')
# the sequence is made of nucleotides: A, C, G, U or T
# the sequence is stored in a string, index starts at 0
class RNA:
    
    # The instantiation operation (“calling” a class object) creates an empty object.
    # Many classes like to create objects with instances customized to a specific initial state.
    # Therefore a class may define a special method named __init__() with or without arguments.

    # initialize with name, id, species, and sequence
    def __init__( self, name, id, species, sequence ):
        self._name = name
        self._id = id
        self._species = species
        # convert the sequence to uppercase
        # change T into U
        self._sequence = sequence.upper().replace( 'T', 'U' )
        # make sure it is made of ACGU only
        is_nucleotides = [c in ['A','C','G','U'] for c in self._sequence]
        if not all( is_nucleotides ):
            fault_position = is_nucleotides.index( False )
            print( 'call to RNA.__init__( sequence = "' + sequence + '" ) illegal nucleotide.' )
            print( ( 34 + fault_position ) * ' ' + '^' )
            exit( 1 )
        
    # Often, the first argument of a method is called self.
    # This is nothing more than a convention: the name self has absolutely no special meaning to Python.
    # Note, however, that by not following the convention your code may be less readable
    #   to other Python programmers, and it is also conceivable that a class browser program
    #   might be written that relies upon such a convention.

    # attribute getters
    def get_name( self ):     return self._name
    def get_id( self ):       return self._id
    def get_species( self ):  return self._species
    def get_sequence( self ): return self._sequence

    # calculated getters
    def __len__( self ): return len( self._sequence )
    
    # Called to implement the built-in function len().
    # Should return the length of the object, an integer >= 0.
    # len( s ), where s is a string, calls __len__ defined for strings.
    
    # sub sequences; use outside world indexing system 1 to n, whereas
    #    internal indexing system of a string is 0 to n-1 for a string of n characters.
    def get_sub_sequence( self, start = 1, end = None ):
        if end is None: end = len( self )
        return self._sequence[start-1:end]
    def get_complement( self, start = 1, end = None ):
        if end is None: end = len( self )
        # ''.join( l ) ) converts list l in string (l must be a list of strings)
        return ''.join( [RNA.complement( x ) for x in self.get_sub_sequence( start, end )] )
    def get_reverse_complement( self, start = 1, end = None ):
        if end is None: end = len( self )
        # s[::-1] reverses string s
        return self.get_complement( start, end )[::-1]
    
    # When you define a function, you can specify a default value for each parameter.
    # To specify default values for parameters, you use the following syntax: parameter = value
    
    # None is frequently used to represent the absence of a value,
    #   as when default arguments are not passed to a function.

    # Python-based rich operations
    
    # convertion to a string using the Fasta format
    #   ex) >name id species name
    #       sequence
    def __str__( self ):
        return ">" + self._name + " " + self._id + " " + self._species + " " + self._name + "\n" + self._sequence

    # Called by str( object ) and the built-in functions format() and print() to compute
    #   the “informal” or nicely printable string representation of an object.
    #   The return value must be a string object.
    
    # determine equality
    def __eq__( self, other ): return self._id is other.get_id()
    
    # __eq__ is so-called a “rich comparison” method.
    # x == y calls x.__eq__( y ).

    # utilities

    # dictionary for base complementarity (class variable)
    _COMPLEMENT = { 'A':'U', 'C':'G', 'U':'A', 'G':'C', 'T':'A' }
    
    # Generally speaking, instance variables are for data unique to each instance,
    #   like _name, _id, _species, and _sequence, and class variables are for
    #   attributes and methods shared by all instances of the class
    
    # returns the complement of a base (class method)
    def complement( base ):
        try: return RNA._COMPLEMENT[base.upper()]
        except:
            print( "call to RNA.complement( base = '" + base + "' ); base must be A, C, G, U or T" )
            exit( 1 )

# testing RNA.py

In [2]:
# class variable and methods are invoked by using the class name in the dot notation.
myRNA1 = RNA( "myRNA1", "Id1", "Homo sapiens", "UGAGGUAGUAGGUUGUAUAGUU" )
print( str( myRNA1 ) + '(' + str( len( myRNA1 ) ) + ' nts)' )
myRNA2 = RNA( "myRNA2", "Id2", "Mus musculus", "acacgtaacacgtgtcgcatactacacgtcattactacattttacac" )
print( str( myRNA2 ) + '(' + str( len( myRNA2 ) ) + ' nts)' )
print( myRNA2.get_sub_sequence( 1, 10 ) )
print( myRNA1 == myRNA2 )
print( myRNA2.get_sequence() )
print( myRNA2.get_complement() )
print( myRNA2.get_complement( 2 ) )
print( myRNA2.get_complement( 3, 5 ) )
print( myRNA2.get_reverse_complement() )
print( RNA.complement( 'a' ) )
myRNA3 = RNA( "myRNA3", "Id3", "Homo sapiens", "caguacauaucggauacaucnauucauuacgaucgaugca") # illegal n character in sequence

>myRNA1 Id1 Homo sapiens myRNA1
UGAGGUAGUAGGUUGUAUAGUU(22 nts)
>myRNA2 Id2 Mus musculus myRNA2
ACACGUAACACGUGUCGCAUACUACACGUCAUUACUACAUUUUACAC(47 nts)
ACACGUAACA
False
ACACGUAACACGUGUCGCAUACUACACGUCAUUACUACAUUUUACAC
UGUGCAUUGUGCACAGCGUAUGAUGUGCAGUAAUGAUGUAAAAUGUG
GUGCAUUGUGCACAGCGUAUGAUGUGCAGUAAUGAUGUAAAAUGUG
UGC
GUGUAAAAUGUAGUAAUGACGUGUAGUAUGCGACACGUGUUACGUGU
U
call to RNA.__init__( sequence = "caguacauaucggauacaucnauucauuacgaucgaugca" ) illegal nucleotide.
                                                      ^


# class MiR

In [4]:
###############
# class Mir #
###############
# is an RNA, the mature product of a miRNA gene
# has a seed defined by nucleotides 2 to 8 (5'->3')

# import the RNA class (a Mir is an RNA)
from RNA import RNA
class Mir( RNA ):

    # seed constants
    _seed_length = 7
    _seed_g1 = 1
    _seed_g2 = 2
    _seed_g5 = 5
    _seed_g8 = 8

    # other getters

    # seed is defined by 7 nucleotides, g2 to g8
    def get_seed( self ):
        return self.get_sub_sequence( Mir._seed_g2, Mir._seed_g8 )

    # reverse seed complement of various length between start and end
    # ex) for the reverse complement of the first 4 nucleotides in the seed
    #       always starts at g2
    def get_seed_reverse_complement( self, length = 4 ):
        if length < 1 or length > Mir._seed_length:
            print( 'call to get_seed_reverse_complement( length = ' + str( length ) + ' ) length must be between 1 and ' + str( Mir._seed_length ) )
            exit( 1 )
        return self.get_reverse_complement( Mir._seed_g2, Mir._seed_g1 + length )

# testing MiR

In [5]:
let_7a_5p = Mir( "let-7a-5p", "MIMAT0000062", "Home sapiens", "UGAGGUAGUAGGUUGUAUAGUU" )
print( let_7a_5p )
print( let_7a_5p.get_seed() )
print( 'Seed RC: ', let_7a_5p.get_seed_reverse_complement() )
print( 'complete seed RC: ', let_7a_5p.get_seed_reverse_complement( 7 ) )
print( 'illegal call: ', let_7a_5p.get_seed_reverse_complement( 10 ) )

>let-7a-5p MIMAT0000062 Home sapiens let-7a-5p
UGAGGUAGUAGGUUGUAUAGUU
GAGGUAG
Seed RC:  CCUC
complete seed RC:  CUACCUC
call to get_seed_reverse_complement( length = 10 ) length must be between 1 and 7
illegal call:  CUACUACCUC


# class Kmer

In [6]:
##############
# class Kmer #
##############
# is an RNA, the product of RNAseq
# has a copy number (count)

# import the RNA class (a Kmer is an RNA)
from RNA import RNA
class Kmer( RNA ):

    # initialize has an additional argument for the copy number
    def __init__( self, name, id, species, sequence, count ):
        super().__init__( name, id, species, sequence )
        self._count = count

    # additional getter
    def get_count( self ): return self._count

    # specific __str__
    def __str__( self ):
        return self._sequence + '\t' + self._id + '\t' + str( self._count )

# testing Kmer

In [7]:
kmer2 = Kmer( "kmer", "2", "Home sapiens", "CAGGACTCCAATATAGAGATAAGTTAATGTC", 93 )
print( kmer2.get_count() )
print( kmer2 )

93
CAGGACUCCAAUAUAGAGAUAAGUUAAUGUC	2	93


# MiRBaseFastaReader

In [12]:
# reader object to read a miRBase miRNA Fasta formatted file
# assume no error in the file
# syntax of the file:
#   >miR-9500 MIMAT0035542 Homo sapiens miR-9500
#   AAGGGAAGAUGGUGACCAC
#   beginning of an entry starts with the > character
#   on the header line, followed by miRNA name, id, species,
#   and name again
#   the next line has the mature miRNA sequence
# the object works as an iterator over the file
# the __next__ function assures the iteration
# Python requires the __iter__ function, returning the instance
# the class iterates by returning a Mir instance for each
# miRNA entry in the input file
class MiRBaseFastaReader:

    # initialize an instance by file name:
    #   1) open the file in read mode
    #   2) read the first line
    def __init__( self, file_name ):
        self._file = open( file_name, "r" )
        self._line = self._file.readline()

    # assumes the attribute _line contains the first line
    # of the next miRNA in the file
    # After the last object, two situations can happen:
    #   1) the file ends by a blank line; _line == ""
    #   2) readline past the last line returns None
    def __next__( self ):
        # detect the end of file
        if self._line is not None and self._line != "":
            # _line contains text (header of a miRNA if no error in the file)
            # just check the first character of the header to be sure!
            if self._line[0] == ">":
                # miRNA entry, so:
                #   1) parse the first (header) line
                #   2) split it with seperator space, resulting in a list of 5 substrings:
                #        ex) ['>miR-9500', 'MIMAT0035542', 'Homo', 'sapiens', 'miR-9500\n']
                #      name in split_header[0][1:]; where [1:] is used to remove the >
                #      id in split_header[1]
                #      species in split_header[2]+split_header[3]; where + concatenates 2 strings
                #      duplicated name in split[4]; ignore it
                #   3) prepare _line for the next iteration
                split_header = self._line.split( sep = " ", maxsplit = 5 )
                # read the sequence on the second line
                sequence = self._file.readline()
                if sequence[-1] == '\n': sequence = sequence[:-1]
                # prepare the iterator for the next entry, by assigning _line to the next line
                self._line = self._file.readline()
                # return a Mir object based on the information read in the file
                return Mir( split_header[0][1:], split_header[1], split_header[2] + split_header[3], sequence )
            else:
                # if the line is not empty and the first character in the header is not >
                # then it is an error in the Fasta file, so report it and exit the program
                print( "Fasta file illegal format: " + self._line )
                exit( 1 )
        # if end of file, stop the iteration by raising
        # the StopIteration exception
        raise StopIteration

    # needed for an iterable class
    def __iter__( self ):
        return self

# testing MiRBaseFastaReader

In [13]:
# create a MiRBaseFastaReader instance for the mirnas.fa file
homo_sapiens_mirnas = MiRBaseFastaReader( "mirnas.fa" )
# the instance is iterable
for mirna in homo_sapiens_mirnas: print( mirna )

>let-7a-5p MIMAT0000062 Homosapiens let-7a-5p
UGAGGUAGUAGGUUGUAUAGUU
>let-7a-3p MIMAT0004481 Homosapiens let-7a-3p
CUAUACAAUCUACUGUCUUUC
>let-7a-2-3p MIMAT0010195 Homosapiens let-7a-2-3p
CUGUACAGCCUCCUAGCUUUCC
>let-7b-5p MIMAT0000063 Homosapiens let-7b-5p
UGAGGUAGUAGGUUGUGUGGUU
>let-7b-3p MIMAT0004482 Homosapiens let-7b-3p
CUAUACAACCUACUGCCUUCCC
>let-7c-5p MIMAT0000064 Homosapiens let-7c-5p
UGAGGUAGUAGGUUGUAUGGUU
>let-7c-3p MIMAT0026472 Homosapiens let-7c-3p
CUGUACAACCUUCUAGCUUUCC
>let-7d-5p MIMAT0000065 Homosapiens let-7d-5p
AGAGGUAGUAGGUUGCAUAGUU
>let-7d-3p MIMAT0004484 Homosapiens let-7d-3p
CUAUACGACCUGCUGCCUUUCU
>let-7e-5p MIMAT0000066 Homosapiens let-7e-5p
UGAGGUAGGAGGUUGUAUAGUU
>let-7e-3p MIMAT0004485 Homosapiens let-7e-3p
CUAUACGGCCUCCUAGCUUUCC
>let-7f-5p MIMAT0000067 Homosapiens let-7f-5p
UGAGGUAGUAGAUUGUAUAGUU
>let-7f-1-3p MIMAT0004486 Homosapiens let-7f-1-3p
CUAUACAAUCUAUUGCCUUCCC
>let-7f-2-3p MIMAT0004487 Homosapiens let-7f-2-3p
CUAUACAGUCUACUGUCUUUCC
>miR-15a-5p MIMAT000006

AGACUGACCUUCAACCCCACAG
>miR-6856-5p MIMAT0027612 Homosapiens miR-6856-5p
AAGAGAGGAGCAGUGGUGCUGUGG
>miR-6856-3p MIMAT0027613 Homosapiens miR-6856-3p
UACAGCCCUGUGAUCUUUCCAG
>miR-6857-5p MIMAT0027614 Homosapiens miR-6857-5p
UUGGGGAUUGGGUCAGGCCAGU
>miR-6857-3p MIMAT0027615 Homosapiens miR-6857-3p
UGACUGAGCUUCUCCCCACAG
>miR-6858-5p MIMAT0027616 Homosapiens miR-6858-5p
GUGAGGAGGGGCUGGCAGGGAC
>miR-6858-3p MIMAT0027617 Homosapiens miR-6858-3p
CAGCCAGCCCCUGCUCACCCCU
>miR-6859-5p MIMAT0027618 Homosapiens miR-6859-5p
GAGAGGAACAUGGGCUCAGGACA
>miR-6859-3p MIMAT0027619 Homosapiens miR-6859-3p
UGACCCCCAUGUCGCCUCUGUAG
>miR-6769b-5p MIMAT0027620 Homosapiens miR-6769b-5p
UGGUGGGUGGGGAGGAGAAGUGC
>miR-6769b-3p MIMAT0027621 Homosapiens miR-6769b-3p
CCCUCUCUGUCCCACCCAUAG
>miR-6860 MIMAT0027622 Homosapiens miR-6860
ACUGGGCAGGGCUGUGGUGAGU
>miR-6861-5p MIMAT0027623 Homosapiens miR-6861-5p
ACUGGGUAGGUGGGGCUCCAGG
>miR-6861-3p MIMAT0027624 Homosapiens miR-6861-3p
UGGACCUCUCCUCCCCAG
>miR-6862-5p MIMAT0027625 Homos

# TSVKmerReader

In [14]:
# reader object to read a TSV kmer data file
# assume no error in the file
# syntax of the file:
#   Seq	Id	Count
#   CAGGACTCCAATATAGAGATAAGTTAATGTC	2	93
#   ...
#   the first line must be skipped
#   the each entry starts with the kmer sequence
#   followed by the kmer number and the count 
#   the separator is the tab character (\t in Python)
# the object works as an iterator over the file
# the __next__ function assures the iteration
# Python requires the __iter__ function, returning the instance
# the class iterates by returning a Kmer instance for each
# entry in the input file
class TSVKmerReader:

    # initialize an instance by file name:
    #   1) open the file in read mode
    #   2) read the two first lines
    def __init__( self, file_name ):
        self._file = open( file_name, "r" )
        self._file.readline()
        self._line = self._file.readline()

    # the attribute _line contains the data line
    # of the next kmer in the file
    # After the last object, two situations can happen:
    #   1) the file ends by a blank line; _line == ""
    #   2) readline past the last line returns None
    def __next__( self ):
        # detect the end of file
        if self._line is not None and self._line != "":
            # _line contains kmer, so:
            #   1) parse the line, split with separator \t gives 3 substrings
            #        ex) ['CAGGACTCCAATATAGAGATAAGTTAATGTC','2','93\n']
            #    sequence in split_line[0]
            #    kmer id in split_line[1]
            #    kmer count in split_line[2]
            #   2) prepare _line for the next iteration
            split_line = self._line.split( sep = "\t", maxsplit = 3 )
            id = split_line[1]
            count = split_line[2]
            if count[-1] == '\n': count = count[:-1]
            # prepare the iterator for the next entry, by assigning _line to the next line
            self._line = self._file.readline()
            # return a Kmer object based on the information read in the file
            return Kmer( 'kmer', id, 'Homo sapiens', split_line[0], count )
        else:
            # if the line is not empty
            # then it is an error in the TSV file, so report it and exit the program
            if self._line != "":
                print( "TSV file illegal format: " + self._line )
                exit( 1 )
        # if end of file, close the file and
        # stop the iteration by raising
        # the StopIteration exception
        self._file.close()
        raise StopIteration

    # needed for an iterable class
    def __iter__( self ):
        return self

# testing TSVKmerReader

In [15]:
# create a TSVKmerReader instance for the kmers.tsv file
kmers = TSVKmerReader( "kmers.tsv" )
# the instance is iterable
for kmer in kmers: print( kmer )

AAAAAAAAAAAAAAAAAAAAAAAAAAAAAAA	1	113422
CAGGACUCCAAUAUAGAGAUAAGUUAAUGUC	2	93
UAUGUAAUUGGUUCCAGUGUGAGUCAUUAAA	3	5
GAUAUUUUCGAAAAGUGGGAUUUUUUAAACC	4	88
CUCCAUCUCAGGUAUUAGAAUGAAUGCUUAC	5	7
GGGCUGUUCAAAAUAGCUGGAGCCCCAGACA	6	10
CUUCCAUGGCUGUCCGGAUCGCCGCACUGCA	7	299
GCACCAGGCCUUUCUCUAGAAGUCCUGAGAC	8	128
AGCUGACUGGGUGCAAAAAGGUUGUCAAUAA	9	13
AAGUCAGGAGUCACAAGGAUCUACCAAAUAA	10	23
GCAUCUGCACACACACAGGGAACUUUAAGCA	11	22
AUCAAUCGACUCAGAUGAUCAGUUUUGGUAG	12	252
CAAAGUUUGAUCUGGAAUGUUGGGACUGCAG	13	9
AAACAUCCACCUGAAAUCCUACUACCCCCUC	14	48
CAAAGCAGGACUUUCCCAGUAAAAUCACAGC	15	21
GGCCUGGGCUGGAAACAGCUCUGUGUGUGAA	16	330
ACAAAAAAAGGGGAGUAUGGGUAAGGACCCU	17	5
GCCGGCCUCGUAGUCCAGGAAGACGCCGAGC	18	24
GAGUGGACAAUGCUGCCAGUGAAAAGAACAA	19	129
AAGGCAACUGACCCAAACCACCAUGUGCCUG	20	336
CCCCACACCUGCCGGGCUUUGCCUUUGCUCC	21	6
AAGGGAGGGAAGGAAGGAAGGAAGGAGGGAA	22	22
ACUGAUGAUAUUGUCCUAAUGGCACAAACGC	23	889
AUAAGUGAAACCAUGAAAUGUCUCAGAAGAC	24	6
UGCAAUAUCCCCAAACACUCACCAGCCCGCA	25	6
AACCUUGACUCAUGAAAAGCUAAGUUGUUGU	26	7
AUUAUGCAUCAUUAUUUG

AAGUGCUCAUGACAUUUUGCCAAAUUUUCUA	2288	19
CCUUUGGAAUCCUUGGCGUGCCCAACUGCAG	2289	5
AAAAAGAUAGUCUCUUCAAAAAUAAUGCUGG	2290	14
AUGAUAUUGGGAGAGAAAAACUAUAGGAUGC	2291	9
GAAAUCCAUGAGUUGGUUUUUGGAAUGGGAA	2292	10
AGCCUGCUCCCCCACCCCCUCCCCAGGGGAG	2293	10
GUAUUUUUUUCUGACUUUCACUCUAAGGAUA	2294	66
UCAGCAGUACAGUUCACUGGGCAAGAAGAAA	2295	12
AGUGCAGUCCACGGUGUGAAACAUGAGGGAG	2296	246
AACAGUGCAUAAAGUGGCAGCUAACACGCAG	2297	5
AGGCCGAGAUCAUCAGAAGGCCACCCUGUAG	2298	5
CCCCUCCUGCCACGUCUCCACGGUCACCACC	2299	5
GAGGCUGACAAAUCCAGUUUCUCAGAAAGAA	2300	15
AGGAACAAGCAAGAAGGUGGGAGGAGUAAGG	2301	13
GAUUGUGCCACUGCACUCCAGCCUGGGUGAA	2302	34
GAGCCUCCAAAAACAGCACACACUUUGUCCA	2303	88
ACACAGCCUACCCCACUAUCAACAUCCUGUA	2304	19
CAGGCGCAGGCCAGCAUCUCUCACUUCUUCC	2305	3348
CUGGUCCUUCACGGGAAAUGUGACUUUGGAC	2306	5
UGCAUGUCAUAUCAUGCAUCACUGGAAUACA	2307	21
UCUACCCCGCAUCUCUCAGGCGCUAAGCUCA	2308	265
GGAACCUGAGCCCCGUUUGUUCCACCCCAGA	2309	38
CACCGAUCUCACUGGGUCACUGUGUAAAGCA	2310	6
CAAGCAAUGUGGGAAAGCCUUCAGAGCUGCC	2311	11
AAGUAAAUAAAAUUAAAAUUCAGCUCCUCAG	2312	8
CAGA

# Gale_Shapley

In [16]:
# return whether there is a free shaver
def exist_free_shaver( shavers_matches, shavers, solos ):
    return (len( shavers_matches ) + len( solos )) < len( shavers )

# return the next free shaver, or None if there is no such
def free_shaver( shavers_matches, shavers, solos ):
    for i in range( len( shavers ) ): # iterate through the shavers
        if( not i in shavers_matches and not i in solos ): # if a shaver is not engaged
            if( len( shavers[i] ) > 0 ):
                print( 'next free shaver', i, shavers[i] ) # announce him
                return (i, solos) # return him
            else:
                print( i, 'becomes solo' )
                solos.add( i )
    return (None, solos) # return None if no shaver were free

# return the next highest preference of shaver s
def highest_not_proposed( shavers, s ):
    res = shavers[s][0] # save the result
    print( 'will propose lassie', res ) # announce who she is
    return res # return her

# return whether lassie w is not engaged
def is_free( lassies_matches, w ):
    res = not w in lassies_matches # save the result: w is not in the engaged lassies
    if res: print( 'lassie', w, 'is free' ) # if she is free, announce her
    else: print( 'lassie', w, 'is already engaged to', lassies_matches[w] ) # else signal her matching
    return res # return whether lassie w is engaged or not

# engage shaver s and lassie w
def engage( shavers_matches, lassies_matches, s, w ):
    print( 'shaver', s, 'and lassie', w, 'become engaged' ) # announce the match
    shavers_matches[s] = w # match s with w; insert shaver's match
    lassies_matches[w] = s # match w with s; insert lassie's match
    return( shavers_matches, lassies_matches )

# remove lassie w from shaver s preferences
def remove( shavers, s, w ):
    shavers[s].remove( w ) # remove w in s preferences
    return shavers

# return the spouse of lassie w
def spouse( lassies_matches, w ): return lassies_matches[w]

# return whether lassie w prefers shave s1 over shaver s2
def prefers( lassies, w, s1, s2 ):
    res = lassies[w].index( s1 ) < lassies[w].index( s2 ) # save the result: index of s1 in w'preferences is before index of s2
    if res: print( 'but she prefers', s1 ) # announce her preference to s1
    return res # return the fact!

# free shaver s from previous match
def free( shavers_matches, s ):
    print( s, 'is now free' ) # announce freedom
    shavers_matches.pop( s ) # remove shaver s from shavers' matches
    return shavers_matches

# print the matches at iteration k
def print_matches( shavers, lassies, shavers_matches, lassies_matches, solos, k ):
    print( '----------------------' )
    print( '***** Iteration', k, '*****' )
    for i in range( len( shavers ) ):
        if i in shavers_matches:
            print( 'shaver', i, 'is engaged to lassie', shavers_matches[i] )
    print( '----------------------' )

# make the list of unmatched guys
def unmatched( guys, guys_matches ):
    unmatched_list = []
    for i in range( len( guys ) ):
        if not i in guys_matches:
            unmatched_list.append( i )
    unmatched_list.sort()
    return unmatched_list

# make the list of matchs
def matched( guys_matches ):
    matches_list = []
    for i in guys_matches:
        matches_list.append( (i, guys_matches[i]) )
    matches_list.sort()
    return matches_list

def Gale_Shapley( shavers, lassies ):
    shavers_matches = dict() # dict to maintain shavers' matches
    lassies_matches = dict() # dict to maintain lassies' matches
    solos = set() # set of solos
    k = 0
    while exist_free_shaver( shavers_matches, shavers, solos ): # while exist a free shaver s who still has a lassie to propose to
        k += 1 # iteration number
        s, solos = free_shaver( shavers_matches, shavers, solos ) # assign shaver
        w = highest_not_proposed( shavers, s ) # highest shaver's preference who has not been proposed yet
        if is_free( lassies_matches, w ): # if the lassie is free
            shavers_matches, lassies_matches = engage( shavers_matches, lassies_matches, s, w ) # accept proposal
            shavers = remove( shavers, s, w ) # remove the lassie from shaver's preferences
        else:
            w_spouse = spouse( lassies_matches, w ) # lassie w is already engaged
            if prefers( lassies, w, s, w_spouse ): # but if she prefers the new pretender
                shavers_matches, lassies_matches = engage( shavers_matches, lassies_matches, s, w ) # she engages him
                shavers_matches = free( shavers_matches, w_spouse ) # and frees previous spouse
            else: # otherwise, she stays engaged
                shavers = remove( shavers, s, w ) # new pretender removes lassie w from preferences
        print_matches( shavers, lassies, shavers_matches, lassies_matches, solos, k ) # print the iteration's matches
    matches = matched( shavers_matches )
    shavers_solos = unmatched( shavers, shavers_matches )
    print( 'shavers solo:', shavers_solos )
    lassies_solos = unmatched( lassies, lassies_matches )
    print( 'lassies solos:', lassies_solos )
    return matches

# testing Gale_Shapley

In [17]:
shavers = [[0,2,1],[0,2,1],[0,1,2]] # shavers' preferences
lassies = [[0,1,2],[0,2,1],[1,2,0]] # lassies' preferences

print( '#shavers:', len( shavers ), 'preferences:', shavers )
print( '#lassies:', len( lassies ), 'preferences:', lassies )

matches = Gale_Shapley( shavers, lassies )
print( "Gale-Shapley's matches:" )
print( matches )

#shavers: 3 preferences: [[0, 2, 1], [0, 2, 1], [0, 1, 2]]
#lassies: 3 preferences: [[0, 1, 2], [0, 2, 1], [1, 2, 0]]
next free shaver 0 [0, 2, 1]
will propose lassie 0
lassie 0 is free
shaver 0 and lassie 0 become engaged
----------------------
***** Iteration 1 *****
shaver 0 is engaged to lassie 0
----------------------
next free shaver 1 [0, 2, 1]
will propose lassie 0
lassie 0 is already engaged to 0
----------------------
***** Iteration 2 *****
shaver 0 is engaged to lassie 0
----------------------
next free shaver 1 [2, 1]
will propose lassie 2
lassie 2 is free
shaver 1 and lassie 2 become engaged
----------------------
***** Iteration 3 *****
shaver 0 is engaged to lassie 0
shaver 1 is engaged to lassie 2
----------------------
next free shaver 2 [0, 1, 2]
will propose lassie 0
lassie 0 is already engaged to 0
----------------------
***** Iteration 4 *****
shaver 0 is engaged to lassie 0
shaver 1 is engaged to lassie 2
----------------------
next free shaver 2 [1, 2]
will prop

# Microtargetome

In [23]:
# read the kmers
# create a TSVKmerReader instance for the kmers.tsv file
kmr_file = TSVKmerReader( "kmers_short.tsv" )
# the instance is iterable, create a list of kmers and compute total count
kmers = []
for kmer in kmr_file:
    kmers.append( kmer )

print( 'len( kmers ):', len( kmers ) )

# read the mirs
# create a MiRBaseFastaReader instance for the mirnas.fa file
mir_file = MiRBaseFastaReader( "mirnas_short.fa" )
# the instance is iterable, create a list of mirs and indexes
mirs = []
for mir in mir_file:
    mirs.append( mir )

print( 'len( mirs ):', len( mirs ) )

# computes the attraction based on a seed match region and kmer count
def get_attraction( seed_match, kmer_count ):
    return ( seed_match.count( 'A' ) * 2 + seed_match.count( 'U' ) * 2 + seed_match.count( 'C' ) * 3 + seed_match.count( 'G' ) * 3 ) * kmer_count * 100

# complete the code here
# ...

# your code must generate the shavers and lassies preference vectors

shavers = [[1,2,3],[1,2,3],[2,3,1]]
lassies = [[0, 1, 2], [0, 2, 1], [1, 2, 0]]

# then, simply execute the Gale_Shapley function

matches = Gale_Shapley( shavers, lassies )
print( "Gale-Shapley's matches:" )
print( matches )

len( kmers ): 172
len( mirs ): 166
next free shaver 0 [1, 2, 3]
will propose lassie 1
lassie 1 is free
shaver 0 and lassie 1 become engaged
----------------------
***** Iteration 1 *****
shaver 0 is engaged to lassie 1
----------------------
next free shaver 1 [1, 2, 3]
will propose lassie 1
lassie 1 is already engaged to 0
----------------------
***** Iteration 2 *****
shaver 0 is engaged to lassie 1
----------------------
next free shaver 1 [2, 3]
will propose lassie 2
lassie 2 is free
shaver 1 and lassie 2 become engaged
----------------------
***** Iteration 3 *****
shaver 0 is engaged to lassie 1
shaver 1 is engaged to lassie 2
----------------------
next free shaver 2 [2, 3, 1]
will propose lassie 2
lassie 2 is already engaged to 1
----------------------
***** Iteration 4 *****
shaver 0 is engaged to lassie 1
shaver 1 is engaged to lassie 2
----------------------
next free shaver 2 [3, 1]
will propose lassie 3
lassie 3 is free
shaver 2 and lassie 3 become engaged
----------------